<a href="https://colab.research.google.com/github/danielx1611/ComputerVisionAssignments/blob/main/CameraCalibration.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Camera Calibration with Gradio UI

### Camera Calibration

In [4]:
import numpy as np
import cv2 as cv
import glob
from google.colab.patches import cv2_imshow

def run_calibration():
  # termination criteria
  criteria = (cv.TERM_CRITERIA_EPS + cv.TERM_CRITERIA_MAX_ITER, 30, 0.001) # 30: max_iter , 0.001 : accuracy

  # prepare object points, like (0,0,0), (1,0,0), ..., (6,5,0)
  objp = np.zeros((9*6,3), np.float32) #
  objp[:,:2] = np.mgrid[0:9,0:6].T.reshape(-1,2) # reshaped to [42,2] only take X and Y

  # Arrays to store object points and image points from all the images.
  objpoints = [] # 3d point in real world space [X,Y,Z]
  imgpoints = [] # 2d points in image plane. [X,Y]

  image_dir = Path("/content/images")
  images = list(image_dir.glob("*.jpeg")) + list(image_dir.glob("*.jpg"))

  for fname in images:
    print(1)
    img = cv.imread(fname)
    gray = cv.cvtColor(img, cv.COLOR_BGR2GRAY) # convert color to BlackWhite

    ret, corners = cv.findChessboardCorners(gray, (9,6), None) # ret is set to True if corners are found corners return array of corners found

    if ret == True:
      print(2)
      objpoints.append(objp)

      corners2 = cv.cornerSubPix(gray,corners, (11,11), (-1,-1), criteria)
      imgpoints.append(corners2)
      # cv2.imshow() is disabled in Colab, because it causes Jupyter sessions to crash; import cv2_imshow
      cv.drawChessboardCorners(img, (9,6), corners2, ret)
      cv2_imshow(img)

      # Calibration Part
      ret, mtx, dist, rvecs, tvecs = cv.calibrateCamera(objpoints, imgpoints, gray.shape[::-1], None, None)

      img = cv.imread(fname)
      h,  w = img.shape[:2]
      newcameramtx, roi = cv.getOptimalNewCameraMatrix(mtx, dist, (w,h), 1, (w,h))

      # undistort
      mapx, mapy = cv.initUndistortRectifyMap(mtx, dist, None, newcameramtx, (w,h), 5)
      dst = cv.remap(img, mapx, mapy, cv.INTER_LINEAR)

      # crop the image
      x, y, w, h = roi
      dst = dst[y:y+h, x:x+w]
      cv.imwrite('calibresult.png', dst)

      # Save results into calibration.json
      calibration_data = {
          "K": mtx.tolist(),               # Intrinsic matrix
          "D": dist.tolist(),              # Distortion coefficients
          "R": [cv.Rodrigues(r)[0].tolist() for r in rvecs],  # Convert rvec to rotation matrix
          "t": [t.tolist() for t in tvecs],                   # Translation vectors
          "width": w,
          "height": h
      }

      with open("calibration.json", "w") as f:
          json.dump(calibration_data, f, indent=2)

      mean_error = 0
      for i in range(len(objpoints)):
          imgpoints2, _ = cv.projectPoints(objpoints[i], rvecs[i], tvecs[i], mtx, dist)
          error = cv.norm(imgpoints[i], imgpoints2, cv.NORM_L2)/len(imgpoints2)
          mean_error += error

      print( "total error: {}".format(mean_error/len(objpoints)) )

### Gradio UI

In [6]:
import gradio
import os
import json
from pathlib import Path

image_dir = Path("/content/images")
image_dir.mkdir(parents=True, exist_ok=True)

def save_images(images):
  for image in images:
    destination = image_dir / Path(image.name).name
    with open(image.name, "rb") as src, open(destination, "wb") as dst:
      dst.write(src.read())


def run_visualization():
  calibration_data = None
  with open("calibration.json", "r") as f:
    calibration_data = json.load(f)

  results_visualization(calibration_data)


with gradio.Blocks() as ui:
  gradio.Markdown("### Calibration UI")
  upload = gradio.File(file_types=[".jpeg"])
  upload_button = gradio.Button("Upload Images")
  upload_output = gradio.Textbox(label="Upload status")

  calibrate_button = gradio.Button("Run Calibration")

  visualize_button = gradio.Button("Visualize Results")

  upload_button.click(fn=save_images, inputs=upload, outputs=upload_output)
  calibrate_button.click(fn=run_calibration)
  visualize_button.click(fn=run_visualization)

ui.launch(share=False, inline=True)

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Note: opening Chrome Inspector may crash demo inside Colab notebooks.
* To create a public link, set `share=True` in `launch()`.


<IPython.core.display.Javascript object>

### Results Visualization

In [5]:
def results_visualization(calibration_data):
  pass